<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/model_diagnostics/custom_metrics_demo.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/model_diagnostics/custom_metrics_demo.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

  
<td>
    <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

----

# Model Diagnostics - Custom Metrics Demo

* Measuring model quality is critical to efficiently building models. It is important that the metrics used to measure model quality closely align with the business objectives for the model. Otherwise, slight changes in model quality, as they related to these core objectives, are lost to noise. Custom metrics enables users to measure model quality in terms of their exact business goals. By incorporating custom metrics into workflows, users can:
    * Iterate faster
    * Measure and report on model quality
    * Understand marginal value of additional labels and modeling efforts

    
* Checkout this [notebook](custom_metrics_basics.ipynb) for more details on the metrics types and built in functions for calculating metrics.


## Environment Setup

Install dependencies

In [1]:
!pip install -q "labelbox[data]" \
             scikit-image \
             tensorflow

In [2]:
# Run these if running in a colab notebook
COLAB = "google.colab" in str(get_ipython())

if COLAB:
    !git clone https://github.com/Labelbox/labelbox-python.git
    !cd labelbox-python
    !mv labelbox-python/examples/model_assisted_labeling/*.py .

Import libraries

In [3]:
import sys

sys.path.append('../model_assisted_labeling')

import uuid
import numpy as np
from skimage import measure
import requests
from tqdm import notebook
import requests
import csv
import os

from labelbox.schema.ontology import OntologyBuilder, Tool
from labelbox.data.metrics.group import get_label_pairs
from labelbox.schema.media_type import MediaType
from labelbox import Client, LabelingFrontend, MALPredictionImport
from labelbox.data.metrics import (feature_miou_metric, confusion_matrix_metric,
                                   feature_confusion_matrix_metric)
from labelbox.data.serialization import NDJsonConverter
from labelbox.data.annotation_types import (ScalarMetric, LabelList, Label,
                                            ImageData, MaskData, Mask, Polygon,
                                            Point, Rectangle, ObjectAnnotation,
                                            ClassificationAnnotation,
                                            ClassificationAnswer, Radio)

try:
    from image_model import predict, load_model
except ModuleNotFoundError:
    # !git clone https://github.com/Labelbox/labelbox-python.git
    # !cd labelbox-python && git checkout mea-dev
    # !mv labelbox-python/examples/model_assisted_labeling/*.py .
    raise Exception("You will need to run from the labelbox-python git repo")

Configure client

In [4]:
PROJECT_NAME = "Diagnostics Demo Custom Metrics"
MODEL_NAME = "MSCOCO Mapillary Custom Metrics"
MODEL_VERSION = "0.0.0"

API Key

Provide a valid api key below in order to properly connect to the Labelbox Client.

In [5]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)
load_model()  # initialize Tensorflow Model

In [6]:
# Configure for whatever combination of tools and class names that you would like.
class_mappings = {
    1: {
        "name": 'person',
        "kind": Tool.Type.POLYGON
    },
    2: {
        "name": 'bicycle',
        "kind": Tool.Type.SEGMENTATION,
        'color': 64
    },
    3: {
        "name": 'car',
        "kind": Tool.Type.BBOX
    },
    4: {
        "name": 'motorcycle',
        "kind": Tool.Type.BBOX
    },
    6: {
        "name": 'bus',
        "kind": Tool.Type.POLYGON
    },
    7: {
        "name": 'train',
        "kind": Tool.Type.POLYGON
    },
    8: {
        "name": 'truck',
        "kind": Tool.Type.POLYGON
    },
    10: {
        "name": 'traffic light',
        "kind": Tool.Type.POINT
    },
    11: {
        "name": 'fire hydrant',
        "kind": Tool.Type.BBOX
    },
    13: {
        "name": 'stop sign',
        "kind": Tool.Type.SEGMENTATION,
        'color': 255
    },
    14: {
        "name": 'parking meter',
        "kind": Tool.Type.POINT
    },
    28: {
        "name": 'umbrella',
        "kind": Tool.Type.SEGMENTATION,
        'color': 128
    },
    31: {
        "name": 'handbag',
        "kind": Tool.Type.POINT
    },
}

## Create Predictions
* Loop over data_rows, make predictions, and create ndjson

In [7]:
# --- setup dataset ---
# load mapillary sample
sample_csv_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/master/examples/assets/mapillary_sample.csv"
with requests.get(sample_csv_url, stream=True) as r:
    image_data = [
        row.split(',')
        for row in (line.decode('utf-8') for line in r.iter_lines())
    ]

In [8]:
predictions = LabelList()
for (image_url, external_id) in notebook.tqdm(image_data):
    image = ImageData(url=image_url, external_id=external_id)
    height, width = image.value.shape[:2]
    prediction = predict(np.array([image.im_bytes]),
                         min_score=0.5,
                         height=height,
                         width=width)
    boxes, classes, seg_masks = prediction["boxes"], prediction[
        "class_indices"], prediction["seg_masks"]
    annotations = []
    for box, class_idx, seg in zip(boxes, classes, seg_masks):
        if class_idx in class_mappings:
            class_info = class_mappings.get(class_idx)
            if class_info['kind'] == Tool.Type.POLYGON:
                contours = measure.find_contours(seg, 0.5)
                pts = contours[0].astype(np.int32)
                value = Polygon(points=[
                    Point(x=x, y=y) for x, y in np.roll(pts, 1, axis=-1)
                ])
            elif class_info['kind'] == Tool.Type.BBOX:
                value = Rectangle(start=Point(x=box[1], y=box[0]),
                                  end=Point(x=box[3], y=box[2]))
            elif class_info['kind'] == Tool.Type.POINT:
                value = Point(x=(box[1] + box[3]) / 2.,
                              y=(box[0] + box[2]) / 2.)
            elif class_info['kind'] == Tool.Type.SEGMENTATION:
                value = Mask(mask=MaskData.from_2D_arr(seg *
                                                       class_info['color']),
                             color=(class_info['color'],) * 3)
            else:
                raise ValueError(
                    f"Unsupported kind found. {class_info['kind']}")
            annotations.append(
                ObjectAnnotation(name=class_info['name'], value=value))
    predictions.append(Label(data=image, annotations=annotations))

## Setup a project

In [9]:
tools = []
for target in class_mappings.values():
    tools.append(Tool(tool=target['kind'], name=target["name"]))
ontology_builder = OntologyBuilder(tools=tools)

In [10]:
print(f"Setting up: {PROJECT_NAME}")

project = client.create_project(name=PROJECT_NAME, media_type=MediaType.Image)
editor = next(
    client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))
project.setup(editor, ontology_builder.asdict())

dataset = client.create_dataset(name="Mapillary Diagnostics Demo")
print(f"Dataset Created: {dataset.uid}")
project.datasets.connect(dataset)

## Prepare for upload
* Our local annotations need the following:
    1. signed url for segmentation masks
    2. data rows in labelbox
    3. feature schema ids

In [11]:
signer = lambda _bytes: client.upload_data(content=_bytes, sign=True)
predictions.add_url_to_masks(signer) \
         .add_url_to_data(signer) \
         .add_to_dataset(dataset, client.upload_data)

## **Optional** - Create labels with [Model Assisted Labeling](https://docs.labelbox.com/en/core-concepts/model-assisted-labeling)

* Pre-label image so that we can quickly create ground truth
* Create ground truth data for Model Diagnostics
* Click on link below to label

In [12]:
RUN_MAL = True
if RUN_MAL:
    project.enable_model_assisted_labeling()
    # Convert from annotation types to import format
    ndjson_predictions = NDJsonConverter.serialize(predictions)
    upload_task = MALPredictionImport.create_from_objects(
        client, project.uid, f'mal-import-{uuid.uuid4()}', ndjson_predictions)
    upload_task.wait_until_done()
    print(upload_task.state, '\n')

In [13]:
print(f"https://app.labelbox.com/go-label/{project.uid}")

## Export Labels

We do not support `Skipped` labels and have a limit of **2000**

In [14]:
MAX_LABELS = 2000
labels = [
    l for idx, l in enumerate(project.label_generator()) if idx < MAX_LABELS
]

## Setup Model & Model Run

In [15]:
lb_model = client.create_model(name=MODEL_NAME,
                               ontology_id=project.ontology().uid)
lb_model_run = lb_model.create_model_run(MODEL_VERSION)

Select label ids to upload

In [16]:
lb_model_run.upsert_labels([label.uid for label in labels])

### Compute Metrics
* First get pairs of labels and predictions

* Create helper functions for our metric
* All functions will accept ground truth and prediction annotations

In [17]:
from shapely.ops import cascaded_union


def nearby_cars_iou(ground_truths, predictions, area_threshold=17000):
    """
    Metric to track the iou score for cars that are nearby (determined by pixel size).
    
    This might be useful to investigate why the model poorly when vehicles are nearby.
    We might care a lot about optimizing this metric because our self driving car needs to 
     be aware of its immediate surroundings for safety reasons.
    """
    ground_truths = [gt for gt in ground_truths if gt.name == 'car']
    predictions = [pred for pred in predictions if pred.name == 'car']
    ground_truths = cascaded_union([
        gt.value.shapely
        for gt in ground_truths
        if gt.value.shapely.area > area_threshold
    ])
    predictions = cascaded_union([
        pred.value.shapely
        for pred in predictions
        if pred.value.shapely.area > area_threshold
    ])
    union = ground_truths.union(predictions).area
    # If there is no prediction or label then the score is undefined
    if union == 0:
        return []
    return [
        ScalarMetric(
            value=ground_truths.intersection(predictions).area / union,
            metric_name="iou",
            feature_name="car",
            subclass_name=
            "nearby"  # Doesn't necessarily need to be a subclass in the ontology
        )
    ]

* Compute and sssign each metric to prediction label

In [18]:
# Metric functions expect to be provided labels and predictions that correspond to the same data image/video frame
# This get_label_pairs function uses the datarow id to achieve this.
pairs = get_label_pairs(labels, predictions, filter_mismatch=True)
for (ground_truth, prediction) in pairs.values():
    metrics = []
    metrics.extend(
        feature_miou_metric(ground_truth.annotations, prediction.annotations))
    metrics.extend(
        feature_confusion_matrix_metric(ground_truth.annotations,
                                        prediction.annotations))
    metrics.extend(
        nearby_cars_iou(ground_truth.annotations, prediction.annotations))
    prediction.annotations.extend(metrics)

### Upload to Labelbox

In [19]:
upload_task = lb_model_run.add_predictions(
    f'diagnostics-import-{uuid.uuid4()}',
    NDJsonConverter.serialize(predictions))
upload_task.wait_until_done()
print(upload_task.state)

### Open Model Run

In [20]:
for idx, model_run_data_row in enumerate(lb_model_run.model_run_data_rows()):
    if idx == 5:
        break
    print(model_run_data_row.url)